<a href="https://colab.research.google.com/github/dodoya1/web_scraping/blob/main/Web%E3%82%B9%E3%82%AF%E3%83%AC%E3%82%A4%E3%83%94%E3%83%B3%E3%82%B02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Webページ内のデータを自動抽出_Beautiful Soup

今回は、Udemyの受講者数とレビュー数を自動取得しているサイトから受講者数とレビュー数を自動取得する方法を解説する。

(参考文献)
[【Webスクレイピング超入門】2時間で基礎を完全マスター！PythonによるWebスクレイピング入門 連結版](https://youtu.be/VRFfAeW30qE)

In [ ]:
#ライブラリのインストール
!pip install requests
!pip install BeautifulSoup4

In [11]:
import requests
from bs4 import BeautifulSoup

#スクレイピングしたい対象のURLを指定する
url="https://scraping-for-beginner.herokuapp.com/udemy"
#取得する
res=requests.get(url)

#「html.parser」：HTMLの構造を解析する。
soup = BeautifulSoup(res.text, "html.parser")

#受講者数を得る。
#引数attrsでclassを指定する：attrs={"class":"subscribers"}
subscribers = soup.find_all("p", attrs={"class": "subscribers"})[0]
print(subscribers.text)
n_subscribers=int(subscribers.text.split("：")[1])
print(n_subscribers)

#レビュー数を得る。
reviews = soup.find_all("p", attrs={"class": "reviews"}) [0]
print(reviews.text)
n_reviews = int(reviews.text.split("：")[1])
print(n_reviews)



受講生の数：17567
17567
レビューの数：3058
3058


# ランキングサイト掲載情報を自動取得

今回は、それぞれの観光地において、いくつかのカテゴリ別に評価をし、さらに総合評価をしているサイトを対象としてWebスクレイピングをする方法を解説する。

最終的には、それぞれの観光地におけるカテゴリ別の評価点と総合評価をまとめたCSVファイルを生成し、保存する。

(参考文献)
[【Webスクレイピング超入門】2時間で基礎を完全マスター！PythonによるWebスクレイピング入門 連結版](https://youtu.be/VRFfAeW30qE)

In [ ]:
import requests
from bs4 import BeautifulSoup

#スクレイピングしたい対象のURLを指定する
url="https://scraping-for-beginner.herokuapp.com/ranking/"
#取得する
res=requests.get(url)

#「"html.parser"」とすることで、取得したHTMLを見やすくしてくれる。
soup=BeautifulSoup(res.text,"html.parser")

#取得した全てのデータを格納するための変数。
data=[]

#「u_areaListRankingBox」クラスとは、「おすすめ観光地」の一つ一つをパックにしたクラス。
#タグ名、classを指定し、全ての「おすすめ観光地」を取得する。
spots=soup.find_all("div",attrs={"class": "u_areaListRankingBox"})

#それぞれの「おすすめ観光地」に対して以下の処理を行う。
for spot in spots:
    #「おすすめ観光地」の名前を取得。
    spot_name=spot.find("div",attrs={"class": "u_title"})

    #spanタグ、badgeクラスを指定し、その部分を削除(extract)している。
    spot_name.find("span",attrs={"class": "badge"}).extract()

    #不必要な「\n」(改行マーク)を空白文字に置き換えている。
    spot_name=spot_name.text.replace("\n", "")

    #評点(総合評価点)を取得している。
    eval_num=spot.find("div", attrs={"class": "u_rankBox"}).text
    eval_num=float(eval_num.replace("\n", ""))

    #全てのカテゴリー名(項目名)を取得している。
    categoryItems=spot.find("div", attrs={"class": "u_categoryTipsItem"})
    categoryItems=categoryItems.find_all("dl")

    #各カテゴリーの評価点を格納する変数。
    details={}

    # categoryltem=categoryItems[0]
    for categoryItem in categoryItems:
        #カテゴリー名は、「dt」タグに囲われたテキスト。
        category=categoryItem.dt.text
        #各カテゴリーの評価点は、spanタグに囲われたテキスト。
        rank=float(categoryItem.span.text)

        details[category]=rank

    datum=details
    datum["観光地名"]=spot_name
    datum["評点"]=eval_num
    data.append(datum)
data

In [31]:
#CSVファイルに出力
import pandas as pd

df=pd.DataFrame(data)

df

,楽しさ,人混みの多さ,景色,アクセス,観光地名,評点
0,4.6,4.5,4.9,4.2,観光地 1,4.7
1,4.6,4.5,4.9,4.2,観光地 2,4.7
2,4.5,4.4,4.8,4.1,観光地 3,4.6
3,4.4,4.4,4.8,4.0,観光地 4,4.5
4,4.4,4.3,4.7,4.0,観光地 5,4.5
5,4.3,4.3,4.7,3.9,観光地 6,4.4
6,4.2,4.2,4.6,3.8,観光地 7,4.3
7,4.2,4.2,4.6,3.8,観光地 8,4.3
8,4.1,4.1,4.5,3.7,観光地 9,4.2
9,4.0,4.1,4.4,3.6,観光地 10,4.1


In [37]:
#生成したデータ表のカラムの順番を操作する方法。

#カラムを取得する。
print(df.columns)

#以下のようにして簡単にカラムの順番を変更できる。
df=df[['観光地名', '評点','楽しさ', '人混みの多さ', '景色', 'アクセス']]

#csvファイルとして出力する。(index=Falseとすることで、indexは抜きにしてcsvファイルとして出力する。)
df.to_csv("観光地情報.csv",index=False)

Index(['観光地名', '評点', '楽しさ', '人混みの多さ', '景色', 'アクセス'], dtype='object')


# Webページ上の画像を自動で取得

imgフォルダ内に、指定したURL内に存在する全ての画像が自動で格納される。

(参考文献)
[【Webスクレイピング超入門】2時間で基礎を完全マスター！PythonによるWebスクレイピング入門 連結版](https://youtu.be/VRFfAeW30qE)

In [50]:
!pip install Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
import requests
from bs4 import BeautifulSoup
#画像の保存を行うことのできるライブラリ。
from PIL import Image
import io

#スクレイピングしたい対象のURLを指定する
url="https://scraping-for-beginner.herokuapp.com/image"
#取得する
res=requests.get(url)

#「"html.parser"」とすることで、取得したHTMLを見やすくしてくれる。
soup=BeautifulSoup(res.text,"html.parser")
#「img」タグと指定する。「img」タグ全てを取得する。最初の一つだけ取得したい場合は、「soup.find("img")」とする。
img_tags = soup.find_all("img")

#何個目の画像か数えるための変数
count=0
for img_tag in img_tags:
    count+=1
    root_url = "https://scraping-for-beginner.herokuapp.com"
    #以下は、画像のURLである。
    img_url=root_url + img_tag["src"]

    #指定した画像のURL(img_url)先の画像を開く。
    img=Image.open(io.BytesIO(requests.get(img_url).content))
    #「img/sample.jpg」という名前で画像を保存する。
    img.save(f"img/{count}.jpg")

# スクレイピングを自動で定期実行する方法！

以上のスクレイピングを自動で定期実行する方法は、以下のサイトを参考に。

[Python　スクレイピングを自動で定期実行する！【Schedule】](https://retire-early40.com/python%E3%80%80%E3%82%B9%E3%82%AF%E3%83%AC%E3%82%A4%E3%83%94%E3%83%B3%E3%82%B0%E3%82%92%E8%87%AA%E5%8B%95%E3%81%A7%E5%AE%9A%E6%9C%9F%E5%AE%9F%E8%A1%8C%E3%81%99%E3%82%8B%EF%BC%81%E3%80%90schedule/)